In [1]:
from custom_clss_and_fncs import * 

SyntaxError: invalid syntax (custom_clss_and_fncs.py, line 49)

In [ ]:
NX_fluxes_and_concentrations = nx.read_gpickle("./results/graphs_from_PYG_and_NX/NX_fluxes_and_concentrations.gpickle")
NX_only_concentrations       = nx.read_gpickle("./results/graphs_from_PYG_and_NX/NX_only_concentrations.gpickle")
pyg_graph_FplusC      = from_networkx(copy.deepcopy(nx.Graph(NX_fluxes_and_concentrations)))
pyg_graph_onlyC       = from_networkx(copy.deepcopy(nx.Graph(NX_only_concentrations)))
assert not pyg_graph_FplusC.is_directed()
assert not pyg_graph_onlyC.has_isolated_nodes()

In [ ]:
oversampled_metabolite_data         = pd.read_parquet("./results/data/oversampled_augmented_metabolite_data.parquet.gzip")#
labels                              = torch.tensor(oversampled_metabolite_data.label).reshape(len(oversampled_metabolite_data.label),1).int()
assert set(labels.unique().numpy()) == set((1,0))

In [ ]:
from torch_geometric.data import Data
from torch.utils.data import RandomSampler

def make_graphs_list(pyg_graph,labels,mask_target:bool=False, target_node:int = 0, mask_number = 0):
    pyg_graph.y = labels
    graphs_list = []
    for i in range(pyg_graph.x.shape[1]):
    
        
        new_pyg_data  = Data(x =  pyg_graph.x[:,i].reshape(pyg_graph.num_nodes, 1),  y = pyg_graph.y[i], 
                            edge_index = pyg_graph.edge_index)
        new_pyg_data.num_classes = 2
        if mask_target:
            new_pyg_data.x[target_node,:] = mask_number
            
        
        graphs_list.append(new_pyg_data) 
    return graphs_list



def make_loader(graphs: list, batch_size: int  =8*32):

    sampler_train_set = RandomSampler(
        graphs,
        #num_samples= num_samples, #params["training"]["sampler_num_samples"],  # Genera un muestreo del grafo
        replacement=False,  # con repeticion de muestras
    )
    return DataLoader(graphs, batch_size=batch_size, sampler = sampler_train_set,  drop_last=True)

graphs_list_only_concentrations       = make_graphs_list(pyg_graph_onlyC,labels)
graphs_list_fluxes_and_concentrations = make_graphs_list(pyg_graph_FplusC,labels)


loader_onlyC  = make_loader(graphs_list_only_concentrations)
loader_FplusC = make_loader(graphs_list_fluxes_and_concentrations)


In [ ]:
def plot_classsifiers(X, y):
    

    #clf1 = LogisticRegression()
    #clf2 = DecisionTreeClassifier()
    clf3 = RandomForestClassifier()
    clf4 = SVC(gamma='auto')
    clf5 = GaussianNB()
    clf6 = MLPClassifier()

    gs = gridspec.GridSpec(3, 2)
    fig = plt.figure(figsize=(14,7))
    labels = ['Random Forest', 'SVM', 'Naive Bayes', 'Neural Network']
    for clf, lab, grd in zip([clf3, clf4, clf5, clf6],
                            labels,
                            [(0,0), (0,1), (1,0), (1,1)]):
        clf.fit(X, y)
        ax = plt.subplot(gs[grd[0], grd[1]])
        fig = plot_decision_regions(X=X, y=y, clf=clf, legend=2)
        plt.title(lab)
    plt.show()
    
    
    
def train_classifiers_with_dataloader(loader):   
    
    contatenated = torch.Tensor()
    labels_from_loader       = torch.Tensor()

    for data in loader:  # Iterate in batches over the training dataset.
            #data.to('cuda')
            #out = model(data.x, data.edge_index, data.batch)
            reshaped_batch     = data.x.reshape(data.y.shape[0], -1)
            contatenated       = torch.cat((contatenated,reshaped_batch),0)        
            labels_from_loader = torch.cat((labels_from_loader, data.y),0)
            

    non_zero_cols =  np.sum(contatenated.numpy() , 0) !=0
    X_from_loader = contatenated[:,non_zero_cols]

    reducer = umap.UMAP()
    embedding = reducer.fit_transform(X_from_loader)
    plot_classsifiers(embedding, labels_from_loader.to(int).numpy())
    plt.show()

In [ ]:
train_classifiers_with_dataloader(loader_onlyC)
train_classifiers_with_dataloader(loader_FplusC)

In [ ]:


target_node = "phe_L_c"
assert list(NX_only_concentrations.nodes).index(target_node) == list(NX_fluxes_and_concentrations.nodes).index(target_node)
phe_L_c_idx = list(NX_only_concentrations.nodes).index(target_node)


graphs_list_only_concentrations_MASKED       = make_graphs_list(pyg_graph_onlyC,labels, mask_target = True, target_node = phe_L_c_idx)
graphs_list_fluxes_and_concentrations_MASKED = make_graphs_list(pyg_graph_FplusC,labels, mask_target = True, target_node = phe_L_c_idx)


loader_onlyC_MASKED  = make_loader(graphs_list_only_concentrations_MASKED)
loader_FplusC_MASKED = make_loader(graphs_list_fluxes_and_concentrations_MASKED)


In [ ]:
train_classifiers_with_dataloader(loader_onlyC_MASKED)
train_classifiers_with_dataloader(loader_FplusC_MASKED)